In [ ]:
import sys
import os
import nest_asyncio
nest_asyncio.apply()

# Add the parent directory to the system path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from jarjarquant import Jarjarquant
jjq = Jarjarquant()

In [ ]:
from ib_async import IB, util, Index
ib = IB()

# Connect to the IB Gateway or TWS
await ib.connectAsync('127.0.0.1', 7496, clientId=1)

contract = Index('SPX', 'CBOE')

# Request historical implied volatility data
bars = await ib.reqHistoricalDataAsync(
    contract,
    endDateTime='',  # current time
    durationStr='68 Y',  # Duration of data, e.g., '1 M' for 1 month
    barSizeSetting='1 day',  # Bar size, e.g., '1 day'
    whatToShow='TRADES',
    useRTH=True,
    formatDate=1
)

# Convert bars to a DataFrame and display
df = util.df(bars)
# Rename 'open', 'high', 'low', 'close' columns to 'Open', 'High', 'Low', 'Close'
# df.rename(columns={'open': 'Open', 'high': 'High',
#             'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

# Disconnect from IB
ib.disconnect()

In [ ]:
df

In [ ]:
# Use the get_tws_ticker method from data gatherer
# Empty end date implies fetching data upto the last available date
jjq.data_gatherer.get_tws_ticker(ticker='BAC', exchange='SMART', currency='USD', end_date='', duration='1 D', bar_size='1 min', what_to_show='TRADES')

In [ ]:
# Get random price samples
# Valid Bar Sizes:
# 1 secs 5 secs	10 secs	15 secs	30 secs
# 1 min	2 mins	3 mins	5 mins	10 mins	15 mins	20 mins	30 mins
# 1 hour 2 hours 3 hours 4 hours 8 hours
# 1 day
# 1 week
# 1 month
data = jjq.data_gatherer.get_random_price_samples_tws(num_tickers_to_sample=1, years_in_sample=1, bar_size='1 day')
data[0].head()  # Display the first few rows of the sampled data

In [ ]:
# Override years_in_sample to get samples less than 1 year by specifying a custom duration
# Duration formnat:
# "<integer> <duration_unit>"
# Valid durations:
# S	Seconds
# D	Day
# W	Week
# M	Month
# Y	Year
# Ensure years_in_sample is an integer and greater than the specified duration - not specifying years_in_sample will handle both these risks
data = jjq.data_gatherer.get_random_price_samples_tws(num_tickers_to_sample=1, bar_size='5 mins', duration='1 D')
data[0].head()  # Display the first few rows of the sampled data

#### Using the EODHD API to fetch data

In [ ]:
import polars as pl

# Empty DataFrame
empty_df = pl.DataFrame()
print(empty_df.is_empty())  # True
print(empty_df.height == 0)  # True

# DataFrame with columns but no rows
empty_with_schema = pl.DataFrame({"a": [], "b": []})
print(empty_with_schema.is_empty())  # True

# Non-empty DataFrame
df = pl.DataFrame({"a": [1, 2], "b": [3, 4]})
print(df.is_empty())  # False